In [4]:
import numpy as np 
import pandas as pd
df=pd.read_csv('../data/retail.csv',parse_dates=["date"])



Data ingestion and quality checks

In [5]:
assert{"store_id","item_id","date","qty_sold"}.issubset(df.columns)
dup = df.duplicated(["store_id","item_id","date"]).sum()
missing = df["qty_sold"].isna().sum()
neg = (df["qty_sold"]<0).sum()
print({"duplicates":dup, "missing_qty":missing, "neg_qty":neg})
# Remove stockout-censored rows when possible
if "stockout_flag" in df:
    df = df[df["stockout_flag"]==0].copy()
# Fill promo/price na
for c in ["on_promo","discount_pct","price"]:
    if c in df: df[c] = df[c].fillna(0)



{'duplicates': np.int64(3), 'missing_qty': np.int64(2), 'neg_qty': np.int64(1)}


Exploratory Data Analysis (EDA)


In [6]:
g = df.groupby(["item_id"])["qty_sold"].apply(lambda s: (s==0).mean()).rename("p_zero")
print(g.describe())


count    4.000000
mean     0.187500
std      0.239357
min      0.000000
25%      0.000000
50%      0.125000
75%      0.312500
max      0.500000
Name: p_zero, dtype: float64


Feature Engineering (Stat + ML)

In [7]:
import pandas as pd

# ---------------------------------------------------------
# Sort properly
# ---------------------------------------------------------
df = df.sort_values(["store_id", "item_id", "date"])

# ---------------------------------------------------------
# Calendar features (always works)
# ---------------------------------------------------------
df["dow"] = df["date"].dt.weekday
df["week_of_year"] = df["date"].dt.isocalendar().week.astype(int)
df["month"] = df["date"].dt.month

# ---------------------------------------------------------
# SAFE Lag features (check data exists first)
# ---------------------------------------------------------
g = df.groupby(["store_id", "item_id"])["qty_sold"]

# Only create lags if enough history exists
if len(df) >= 28:
    df["lag_1"]  = g.shift(1)
    df["lag_7"]  = g.shift(7)
    df["lag_14"] = g.shift(14)
    df["lag_28"] = g.shift(28)
else:
    df["lag_1"] = g.shift(1)
    df["lag_7"] = g.shift(7)
    print("⚠️ Limited history - using shorter lags")

# ---------------------------------------------------------
# SAFE Rolling features
# ---------------------------------------------------------
df["r7_mean"]  = g.shift(1).rolling(7, min_periods=1).mean()
df["r14_mean"] = g.shift(1).rolling(14, min_periods=1).mean()
df["r28_mean"] = g.shift(1).rolling(28, min_periods=3).mean()

df["r7_std"]   = g.shift(1).rolling(7, min_periods=1).std()
df["r14_max"]  = g.shift(1).rolling(14, min_periods=1).max()

# ---------------------------------------------------------
# RELAXED: Only drop rows missing TARGET (not features)
# ---------------------------------------------------------
df = df.dropna(subset=["qty_sold"]).reset_index(drop=True)
print(f"✅ Features ready: {df.shape[0]} rows, {len([c for c in df.columns if 'lag_' in c or 'r' in c])} features")


⚠️ Limited history - using shorter lags
✅ Features ready: 14 rows, 17 features


Modeling Strategy (Hybrid)


In [8]:

from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import GroupKFold
features = [c for c in df.columns if c not in ["qty_sold","date"]]
X, y = df[features], df["qty_sold"]
# Grouped CV by SKU-store to respect time locality across groups
gkf = GroupKFold(n_splits=5)
groups = df["store_id"].astype(str) + "_" + df["item_id"].astype(str)
rf = RandomForestRegressor(n_estimators=400, max_depth=12, random_state=13, n_jobs=-1)
# (Time-series CV is better: for production use expanding windows per group)




 Intermittent Demand: Croston/SBA Implementation


In [9]:

def croston_forecast(y, alpha=0.1, h=4):
    # y: pandas Series (demand), h: horizon (periods)
    import numpy as np
    demand = y.values
    n = len(demand)
    z = demand[demand>0]
    p = np.diff(np.r_[0, np.where(demand>0)[0]])
    if len(z)==0: return np.zeros(h)
    z_hat, p_hat = z[0], p[1] if len(p)>1 else 1
    for i in range(1, len(z)):
        z_hat = alpha*z[i] + (1-alpha)*z_hat
    for i in range(1, len(p)):
        p_hat = alpha*p[i] + (1-alpha)*p_hat
    f = (z_hat/p_hat) * np.ones(h)      # Croston
    return f  # SBA variant multiplies by (1 - alpha/2)


Backtesting & Accuracy Metrics

In [10]:
def mase(y_true, y_pred, y_naive):
    d = np.abs(y_true - y_pred).mean()
    d_naive = np.abs(y_true - y_naive).mean()
    return d/d_naive if d_naive>0 else np.nan



Inventory Optimization Logic

In [11]:
from scipy.stats import norm
import numpy as np

def inventory_policy(forecast, resid_std, on_hand, lead_time, 
                     annual_demand, ordering_cost, unit_cost, holding_rate,
                     service=0.95):  
    z = norm.ppf(service)
    mu_L = forecast[:lead_time].sum()
    sigma_L = resid_std * (lead_time ** 0.5)
    SS = z * sigma_L
    ROP = mu_L + SS
    H = unit_cost * holding_rate
    EOQ = np.sqrt((2 * annual_demand * ordering_cost) / H) if H > 0 else mu_L
    order_qty = max(0, max(EOQ, ROP - on_hand))
    return dict(mu_L=mu_L, sigma_L=sigma_L, SS=SS, ROP=ROP, EOQ=EOQ, order_qty=order_qty)


In [13]:
# ===== STEP 9: PRODUCTION EXPORT (FIXED - No string errors!) =====
import joblib
from sklearn.model_selection import GroupShuffleSplit

# YOUR features - EXCLUDE categorical columns!
exclude_cols = ["qty_sold", "date", "store_id", "item_id"]
features = [c for c in df.columns if c not in exclude_cols]

# ✅ FIX: Select only NUMERIC columns
numeric_features = []
for col in features:
    if df[col].dtype in ['float64', 'int64', 'float32', 'int32']:
        numeric_features.append(col)
    else:
        print(f"⚠️ Skipping non-numeric: {col}")

print(f"✅ Using {len(numeric_features)} numeric features")

# ✅ SAFE conversion
X = df[numeric_features].astype(float)
y = df["qty_sold"]
groups = df["store_id"].astype(str) + "_" + df["item_id"].astype(str)

# Train YOUR model
gss = GroupShuffleSplit(test_size=0.2, random_state=13)
tr_idx, te_idx = next(gss.split(X, y, groups))

rf = RandomForestRegressor(n_estimators=400, max_depth=12, random_state=13, n_jobs=-1)
rf.fit(X.iloc[tr_idx], y.iloc[tr_idx])

# YOUR residuals
resid_std = float(np.std(y.iloc[te_idx] - rf.predict(X.iloc[te_idx])))

# ✅ EXPORT - Your functions already defined above!
artifacts = {
    'model': rf,
    'features': numeric_features,  # Only numeric!
    'resid_std': resid_std,
    'p_zero_stats': g.describe().to_dict()
}

joblib.dump(artifacts, "retail_forecast_model.pkl")
print("✅ STEP 9 COMPLETE | retail_forecast_model.pkl SAVED")
print("🎬 streamlit run dashboard.py")


⚠️ Skipping non-numeric: category
⚠️ Skipping non-numeric: brand
⚠️ Skipping non-numeric: pack_size
⚠️ Skipping non-numeric: city
⚠️ Skipping non-numeric: cluster
✅ Using 23 numeric features
✅ STEP 9 COMPLETE | retail_forecast_model.pkl SAVED
🎬 streamlit run dashboard.py
